In [1]:
import os
import json

dataDir = "../data/"
relationDir = os.path.join(dataDir, "relations")
structureFile = os.path.join(relationDir, "structure_relations.json")

# read file from structureFile
relations = set()
with open(structureFile, "r") as f:
    structureRelations = json.load(f)
    for idx, structureRelation in enumerate(structureRelations):
        if structureRelation[1] == "目录":
            structureRelations[idx][1] = "parent_content"
        elif structureRelation[1] == "前置":
            structureRelations[idx][1] = "prerequisites"
        relations.add(structureRelations[idx][1])


print("# of relations: ", len(relations))
print(relations)

print("# of structure relations: ", len(structureRelations))

# of relations:  4
{'co_presence', 'parent_content', 'include', 'prerequisites'}
# of structure relations:  92622


In [2]:
import networkx as nx

graphDir = os.path.join(dataDir, "graph")


def build_graph(relations):
    graph = nx.DiGraph()
    for relation in relations:
        graph.add_edge(relation[0], relation[2], relation=relation[1])
    return graph


graph = build_graph(structureRelations)
# print # of nodes and edges
print("# of nodes: ", graph.number_of_nodes())
print("# of edges: ", graph.number_of_edges())

# # export graph to json file
# import json

# graphJsonFile = os.path.join(graphDir, "structure_graph.json")
# with open(graphJsonFile, "w") as f:
#     json.dump(nx.node_link_data(graph), f, ensure_ascii=False, indent=4)

# of nodes:  4296
# of edges:  92622


In [5]:
# find all links connected to the node named "Deep Learning"
links = graph.edges(data=True)
print(len(links))
for link in links:
    if link[2]["relation"] == "include":
        print(link)

92622
('1.2 Historical Trends in Deep Learning', 'tensor', {'relation': 'include'})
('1.2 Historical Trends in Deep Learning', 'counter', {'relation': 'include'})
('1.2 Historical Trends in Deep Learning', 'data generating distribution', {'relation': 'include'})
('1.2 Historical Trends in Deep Learning', '2-D slice of tensor', {'relation': 'include'})
('1.2 Historical Trends in Deep Learning', 'logistic sigmoid', {'relation': 'include'})
('1.2 Historical Trends in Deep Learning', 'variance', {'relation': 'include'})
('1.2 Historical Trends in Deep Learning', 'concepts', {'relation': 'include'})
('1.2 Historical Trends in Deep Learning', 'formal mathematical rules', {'relation': 'include'})
('1.2 Historical Trends in Deep Learning', 'gathering knowledge', {'relation': 'include'})
('1.2 Historical Trends in Deep Learning', 'inventors', {'relation': 'include'})
('1.2 Historical Trends in Deep Learning', 'column of matrix', {'relation': 'include'})
('1.2 Historical Trends in Deep Learning'